In [1]:
train = True

In [2]:
! ls vineyards-1/valid

images	labels	labels.cache


In [3]:
import gdown
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patches
import pandas as pd
import json
from tqdm import tqdm
import os
from PIL import Image

import albumentations as A
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn

import ultralytics
from ultralytics import YOLO

from clearml import Task, Logger

In [4]:
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=J35TDHSFXOCRAW5QIRR5YF4EN0OA8R
%env CLEARML_API_SECRET_KEY=BKihDZnjDtPZzlBwNpsE6LY6r6tpJqZIbZuKiECKAu2pB78knuK6voRWtcWoizSq_J8

env: CLEARML_WEB_HOST=https://app.clear.ml/
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=J35TDHSFXOCRAW5QIRR5YF4EN0OA8R
env: CLEARML_API_SECRET_KEY=BKihDZnjDtPZzlBwNpsE6LY6r6tpJqZIbZuKiECKAu2pB78knuK6voRWtcWoizSq_J8


In [7]:
pip freeze | grep ultralytics  

ultralytics==8.2.64
ultralytics-thop==2.0.0
Note: you may need to restart the kernel to use updated packages.


In [11]:
! ls /home/comp2/vineyards-1

data.yaml  README.dataset.txt  README.roboflow.txt  test  train  valid


In [13]:
dataset_yaml = '/home/comp2/vineyards-1/data.yaml'
train_dir = '/home/comp2/vineyards-1/train/images'
val_dir = '/home/comp2/vineyards-1/valid'
test_dir = '/home/comp2/vineyards-1/test'
save_dir = '/home/comp2/runs/segment'
best_dir = os.path.join(save_dir, 'train22/weights/best.pt') 

In [15]:
! ls /home/comp2/runs/segment/train22/weights/best.pt

/home/comp2/runs/segment/train22/weights/best.pt


In [17]:
ultralytics.checks()

Ultralytics YOLOv8.2.64 🚀 Python-3.12.4 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24084MiB)
Setup complete ✅ (32 CPUs, 125.6 GB RAM, 147.1/862.3 GB disk)


In [21]:
if train:
    task = Task.init(
        project_name='vineyard-yolov8_50_boxes',
        task_name='training-and-inference',
        tags=['YOLOv8-50']
    )

In [23]:
if train:
    model = YOLO('yolov8n-seg.pt', task='segment')
    model.train(data=dataset_yaml, epochs=50)

2024-07-24 14:25:16,836 - clearml.model - INFO - Selected model id: b4c1b8321ce34724a30c730a7f8217ee
Ultralytics YOLOv8.2.64 🚀 Python-3.12.4 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24084MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/home/comp2/vineyards-1/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train23, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, emb

train: Scanning /home/comp2/vineyards-1/train/labels.cache... 363 images, 0 backgrounds, 76 corrupt: 100%|██████████| 363/363 [00:00<?, ?it/s]

train: WARNING ⚠️ /home/comp2/vineyards-1/train/images/16crop_png_jpg.rf.19b6d0b8a2053eec4c60e13065b22b01.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0266]
train: WARNING ⚠️ /home/comp2/vineyards-1/train/images/16crop_png_jpg.rf.1e383aa4d55803d73d15ca4d47263fe1.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0265]
train: WARNING ⚠️ /home/comp2/vineyards-1/train/images/16crop_png_jpg.rf.4a00ac59204d2a82a8bbd2b699aea41f.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0327]
train: WARNING ⚠️ /home/comp2/vineyards-1/train/images/16crop_png_jpg.rf.4fd80c4d1302c8b605b07f3e318b6acf.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0049]
train: WARNING ⚠️ /home/comp2/vineyards-1/train/images/16crop_png_jpg.rf.7da866fcd67426ea14b19af7dc6b7dec.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0277]
train


val: Scanning /home/comp2/vineyards-1/valid/labels.cache... 15 images, 0 backgrounds, 3 corrupt: 100%|██████████| 15/15 [00:00<?, ?it/s]

val: WARNING ⚠️ /home/comp2/vineyards-1/valid/images/20crop_png_jpg.rf.4334fc4b1e5583b8cdf9634dae2d9e94.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0101]
val: WARNING ⚠️ /home/comp2/vineyards-1/valid/images/20crop_png_jpg.rf.b1a03e35a14c48a9e08e0ef08d3f35fe.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.011       1.006]
val: WARNING ⚠️ /home/comp2/vineyards-1/valid/images/20crop_png_jpg.rf.d1618fd32f6be7d916ecc6b41bbe0d55.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.004]


...

      50/50      4.42G      1.123      1.898     0.6667      1.075        621        640: 100%|██████████| 18/18 [00:01<00:00, 15.87it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 20.63it/s]

                   all         12        460      0.932      0.897      0.947      0.634      0.931      0.902      0.944       0.53



50 epochs completed in 0.026 hours.
Optimizer stripped from runs/segment/train23/weights/last.pt, 6.8MB
Optimizer stripped from runs/segment/train23/weights/best.pt, 6.8MB

Validating runs/segment/train23/weights/best.pt...
Ultralytics YOLOv8.2.64 🚀 Python-3.12.4 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24084MiB)
YOLOv8n-seg summary (fused): 195 layers, 3,258,259 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 18.43it/s]


                   all         12        460       0.93      0.897      0.946      0.633      0.929      0.902      0.944       0.53
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/segment/train23


In [25]:
model = YOLO(best_dir)

In [ ]:
# Функция для логирования и вывода боксов
def log_and_print_bboxes(image_path, model):
    image = Image.open(image_path)
    results = model(image)

    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]  # Координаты бокса
            label = box.cls  # Класс объекта
            confidence = box.conf  # Уверенность
    
       
            # Вывод в консоль
            print(f"Image: {image_path}, Box: [{x1}, {y1}, {x2}, {y2}], Class: {label}, Confidence: {confidence}")
    print(f'Количество bounding boxes на данной фотографии, то есть количество найденных виноградников ', len(boxes))


In [ ]:
! ls /home/comp2/vineyards-1/test/images

In [59]:
# Пример логирования и вывода для одного изображения
test_image_path = '/home/comp2/vineyards-1/test/images/10crop_png_jpg.rf.06d8a88b65d376474607a6b8a8d30227.jpg'  # Замените на реальный путь к вашему тестовому изображению
log_and_print_bboxes(test_image_path, model)


0: 640x640 74 vineyards, 2.1ms
Speed: 10.4ms preprocess, 2.1ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)
Image: /home/comp2/vineyards-1/test/images/10crop_png_jpg.rf.06d8a88b65d376474607a6b8a8d30227.jpg, Box: [211.68557739257812, 131.093994140625, 277.9109191894531, 191.99246215820312], Class: tensor([0.], device='cuda:0'), Confidence: tensor([0.9278], device='cuda:0')
Image: /home/comp2/vineyards-1/test/images/10crop_png_jpg.rf.06d8a88b65d376474607a6b8a8d30227.jpg, Box: [328.5447998046875, 181.77035522460938, 421.33837890625, 271.8367004394531], Class: tensor([0.], device='cuda:0'), Confidence: tensor([0.9232], device='cuda:0')
Image: /home/comp2/vineyards-1/test/images/10crop_png_jpg.rf.06d8a88b65d376474607a6b8a8d30227.jpg, Box: [411.46337890625, 207.79815673828125, 480.67962646484375, 272.07293701171875], Class: tensor([0.], device='cuda:0'), Confidence: tensor([0.9107], device='cuda:0')
Image: /home/comp2/vineyards-1/test/images/10crop_png_jpg.rf.06d8a88b65d